In [ ]:
!nvidia-smi

Wed Jul 15 19:17:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip uninstall keras-applications --yes

Uninstalling Keras-Applications-1.0.8:
  Successfully uninstalled Keras-Applications-1.0.8


In [ ]:
!pip install git+https://github.com/keras-team/keras-applications.git@master

  Cloning https://github.com/keras-team/keras-applications.git (to revision master) to /tmp/pip-req-build-ixdbb7rx
  Running command git clone -q https://github.com/keras-team/keras-applications.git /tmp/pip-req-build-ixdbb7rx
  Created wheel for Keras-Applications: filename=Keras_Applications-1.0.8-cp36-none-any.whl size=62000 sha256=91a7b7b3c4524e56cbcaf4521057b7cfe061ce6c82c0cfd34f6b8050ba72c55c
  Stored in directory: /tmp/pip-ephem-wheel-cache-oo1c1nmv/wheels/f8/3c/48/007f19c86cc5d5f276a748f003a63bc19558539eab93afb545
Successfully built Keras-Applications


In [ ]:
from keras.applications import keras_applications
from keras_applications import mobilenet_v3

Using TensorFlow backend.


In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
base_model = mobilenet_v3.MobileNetV3Small(input_shape=(None, None, 3),
                              alpha=1.0,
                              minimalistic=False,
                              include_top=False,
                              weights='imagenet',
                              backend=tf.keras.backend,
                              layers=tf.keras.layers,
                              models=tf.keras.models,
                              utils=tf.keras.utils
                              )

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v3.py:348: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [ ]:
base_model.summary()

In [ ]:
base_model.trainable = False

In [ ]:
out = base_model.predict(np.random.randn(1,320,320,3))
out.shape

(1, 10, 10, 576)

In [ ]:
num_classes = 1

In [ ]:
input = tf.keras.layers.Input(shape=(None, None, 576))
sf = tf.keras.layers.Conv2D(128, kernel_size=1, strides=1)(input)
sf = tf.keras.layers.Activation(mobilenet_v3.hard_swish)(sf)
sf = tf.keras.layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding="same")(sf)
sf = tf.keras.layers.Activation(mobilenet_v3.hard_swish)(sf)
sf = tf.keras.layers.Conv2D(num_classes, kernel_size=1, strides=1)(sf)
sf = tf.keras.layers.Activation('sigmoid')(sf)
sf.shape

TensorShape([None, None, None, 1])

In [ ]:
clf = tf.keras.models.Model(inputs=input, outputs=sf)

In [ ]:
clf.summary()

Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_48 (InputLayer)        [(None, None, None, 576)] 0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, None, None, 128)   73856     
_________________________________________________________________
activation_128 (Activation)  (None, None, None, 128)   0         
_________________________________________________________________
conv2d_transpose_59 (Conv2DT (None, None, None, 64)    73792     
_________________________________________________________________
activation_129 (Activation)  (None, None, None, 64)    0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, None, None, 1)     65        
_________________________________________________________________
activation_130 (Activation)  (None, None, None, 1)     0  

In [ ]:
model = tf.keras.Sequential([base_model, clf])

In [ ]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
MobilenetV3small (Model)     (None, None, None, 576)   939120    
_________________________________________________________________
model_21 (Model)             (None, None, None, 1)     147713    
Total params: 1,086,833
Trainable params: 147,713
Non-trainable params: 939,120
_________________________________________________________________


In [ ]:
out = model.predict(np.random.randn(1,320,320,3))
out.shape

(1, 20, 20, 1)